In [23]:
import cv2
from PIL import Image
import PIL
import os
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import ndimage
import torch
import torch.nn as nn
import numba
from tqdm.notebook import tqdm

In [2]:
TestSet1 = []
for file in os.listdir('Task6/TestSet1'): 
    TestSet1.append(cv2.cvtColor(np.asarray(Image.open(f'Task6/TestSet1/{file}')), cv2.COLOR_BGR2GRAY))

In [53]:
TestSet2 = []
for file in os.listdir('Task6/TestSet2'): 
    TestSet2.append(cv2.cvtColor(np.asarray(Image.open(f'Task6/TestSet2/{file}')), cv2.COLOR_BGR2GRAY))

In [3]:
@numba.njit
def fht(img, upper_border, lower_border):
    n, m = img.shape[0], img.shape[1] 
    result = np.zeros((n, lower_border - upper_border))
    if lower_border - upper_border == 1:
        result[:, 0] = img[:, upper_border]
        return result
    middle = (lower_border + upper_border) // 2
    upper_half = fht(img, upper_border, middle)
    lower_half = fht(img, middle, lower_border)
    for shift in range(lower_border - upper_border):
        for i in range(n):
            result[i, shift] = upper_half[i, shift // 2] + lower_half[(i + shift // 2 + shift % 2) % n, shift // 2]
    return result

def fht_arbitrary_shape(img):
    new_xshape = 2 ** math.ceil(math.log2(img.shape[1])) #добавляем паддинг для того, чтобы размер изображения стал степенью двойки
    new_img = np.pad(img, [[0, 0], 
                          [(new_xshape - img.shape[1]) // 2, (new_xshape - img.shape[1]) // 2 + (new_xshape - img.shape[1]) % 2],
                          ], mode='empty')
    result = fht(new_img, 0, new_xshape)
    return new_img, result

In [4]:
def rotate_image(img):
    x = cv2.Sobel(img,cv2.CV_16S,1,0)  #применяем фильтр Собеля
    y = cv2.Sobel(img,cv2.CV_16S,0,1)  

    absX = cv2.convertScaleAbs(x)
    absY = cv2.convertScaleAbs(y) 
  
    dst = cv2.addWeighted(absX,0.5,absY,0.5,0)  
    
    rot = fht_arbitrary_shape(dst), fht_arbitrary_shape(np.flip(dst, axis=1))
    rotation_is_clockwise = np.argmax([np.var(rot[0][1], axis=0).max(), np.var(rot[1][1], axis=0).max()])
    rotation_angle = math.atan(np.argmax(np.var(rot[rotation_is_clockwise][1]
                                                , axis=0)) / rot[rotation_is_clockwise][1].shape[1]) * 180 / math.pi
    return ndimage.rotate(img, (-1)**rotation_is_clockwise * rotation_angle)

In [47]:
def get_boxes(anchors):
    if len(anchors) == 0:
        return []
    x = anchors[:, :, :, 0]
    y = anchors[:, :, :, 1]
    w, h = np.max(x) - np.min(x), np.max(y) - np.min(y)
    w, h = max(w,h), max(w,h)
    box = [np.min(x), np.min(y), w, h]
    return box

In [48]:
def plot_result(img, boxes):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img)
    for box in boxes:
        plt.plot([box[0], box[0] + box[2], box[0] + box[2], box[0], box[0]], 
                 [box[1], box[1], box[1] + box[3], box[1] + box[3], box[1]])
    plt.show()

In [49]:
def is_square(anchor, eps):
    x = anchor[:, :, 0]
    y = anchor[:, :, 1]
    return abs(1 - (np.max(x) - np.min(x)) / (np.max(y) - np.min(y))) < eps

In [50]:
def make_anchor_clusters(anchors, eps):
    if len(anchors) == 0:
        return []
    clusters = []
    x = anchors[:, :, :, 0]
    y = anchors[:, :, :, 1]
    for i in range(len(anchors)):
        for j in range(i):
            if abs(1 - np.min(x[i]) / np.min(x[j])) < eps or abs(1 - np.min(y[i]) / np.min(y[j])) < eps or \
                abs(1 - (np.max(x[i]) - np.min(x[j])) / (np.max(y[i]) - np.min(y[j]))) < eps:
                clusters.append([i, j])
    for i in range(len(anchors)):
        for j in range(len(clusters)):
            if i in clusters[j]:
                continue
            for k in clusters[j]:
                if abs(1 - np.min(x[i]) / np.min(x[k])) < eps or abs(1 - np.min(y[i]) / np.min(y[k])) < eps or \
                abs(1 - (np.max(x[i]) - np.min(x[k])) / (np.max(y[i]) - np.min(y[k]))) < eps:
                    clusters[j] += [i]
                    break
    return [[anchors[k] for k in cluster] for cluster in clusters]

In [72]:
def find_qr_codes(img):
    img = rotate_image(img)
    thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    anchors = []
    for i, cnt in enumerate(cnts):
        if hierarchy[0][i][2] > 0 and hierarchy[0][i][3] > 0 and hierarchy[0][hierarchy[0][i][2]][2] > 0:
            cnt = cv2.approxPolyDP(cnt, 10, closed=True)
            if cnt.shape[0] == 4:
                anchors.append(cnt)
    clusters = make_anchor_clusters(np.array(anchors)[[is_square(anchor, 5e-1) for anchor in anchors]], 1e-2)
    boxes = []
    for cluster in clusters:
        boxes.append(get_boxes(np.array(cluster)))
    return (img, boxes)

In [ ]:
for i, image in tqdm(list(enumerate(TestSet1))):
    img, boxes = find_qr_codes(image)    
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    for box in boxes:
        plt.plot([box[0], box[0] + box[2], box[0] + box[2], box[0], box[0]], 
                 [box[1], box[1], box[1] + box[3], box[1] + box[3], box[1]])
    plt.savefig(f'Task_6_Results/TestSet1/img_{i}.png')

In [ ]:
for i, image in tqdm(list(enumerate(TestSet2))):
    img, boxes = find_qr_codes(image)    
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    for box in boxes:
        plt.plot([box[0], box[0] + box[2], box[0] + box[2], box[0], box[0]], 
                 [box[1], box[1], box[1] + box[3], box[1] + box[3], box[1]])
    plt.savefig(f'Task_6_Results/TestSet2/img_{i}.png')

Описание алгоритма: <br>
    При помощи одного из реализованных ранее домашних заданий выравниваем изображение вдоль осей <br>
    С помощью opencv находим контуры фигур и иерархию контуров <br>
    Апроксимируем найденные контуры более простыми кривыми и ищем кривые приближенно похожие на квадраты <br>
    Ищем с помощью найденных контуров и иерархии "опорные" точки qr-кодов <br>
    Разбиваем найденные точки по qr-кодам исходя из их положения, если есть 2 и более подходящие точки в данном множестве, отрисовываем bounding-box данного qr-кода <br><br>

В среднем на один прогон изображений из одного из TestSet'ов тратится ~ 7.5 минут. Итого время работы алгоритма ~9.6 секунд на изображение. <br><br>

Для 1 множества имеем 19 правильных срабатываний и 2 неправильных, всего кодов на изображениях 48, т.е. precision ~90.5% и recall ~39.6% <br>
Для 2 множества имеем 12 правильных срабатываний и 14 неправильных, всего кодов на изображениях 48, т.е. precision ~46.2% и recall ~25% <br>
Результат, конечно, далеко не самый лучший, но зато почти своими руками:)